In [20]:
import json
import nltk
import pandas as pd
# conda install -c conda-forge nltk
from nltk.tokenize.toktok import ToktokTokenizer
import spacy 
#conda install -c conda-forge spacy
from datetime import datetime
import tweepy
import re
import string
import unicodedata
from gensim import corpora
# conda install -c conda-forge gensim

### Sentiment analysis
from textblob import TextBlob
# conda install -c conda-forge textblob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# conda install -c conda-forge vaderSentiment

In [2]:
def clean_tweet(tweet): 
    processed_tweet = tweet
    processed_tweet["id"] = tweet['id']
    processed_tweet["user"] = tweet['author_id']
    created_at = datetime.strptime(tweet["created_at"],"%Y-%m-%dT%H:%M:%S.%fZ")
    processed_tweet["created_at"] = tweet['created_at']
    processed_tweet["lang"] = tweet['lang']
    
    if tweet['lang'] != "en":
        processed_tweet["is_en"] = False
    else: 
        processed_tweet["is_en"] = True
            
    return processed_tweet

In [3]:
data_list1 = json.loads(open('/Users/planetyunnn/Desktop/HUSTLE/2022 Spring/COMM 557/Codes/Usable Data/hmlscvd_tweets_0404.json').read())
data_list2 = json.loads(open('/Users/planetyunnn/Desktop/HUSTLE/2022 Spring/COMM 557/Codes/Usable Data/hmlscvd_tweets_0411.json').read())
data_list3 = json.loads(open('/Users/planetyunnn/Desktop/HUSTLE/2022 Spring/COMM 557/Codes/Usable Data/hmlscvd_tweets_0419.json').read())

tweet_data_list = data_list1 + data_list2 + data_list3

len(tweet_data_list)

9170

In [4]:
filtered_data = []
for elem in tweet_data_list: 
    filtered_tweet = clean_tweet(elem)
    filtered_data.append(filtered_tweet)

In [5]:
clean_tweets_json = []
for fd in filtered_data:
    #fd['created_at'] = datetime.strftime(fd['created_at'],"%Y-%m-%dT%H:%M:%S.%fZ")
    clean_tweets_json.append(fd)
with open("hmlscvd_clean.json", "w") as outfile:
    json.dump(clean_tweets_json, outfile, indent=4)

In [6]:
tweet_text = []
for tweet in filtered_data:
    if tweet["is_en"]:
        if tweet.get("is_rt"): 
            tweet_text.append(tweet["rt_text"].replace("\n", " "))
        else:
            tweet_text.append(tweet["text"].replace("\n", " "))# filter all of the raw tweets by turning them into clean_tweet objects
# the filtering is taken care of in the class function
filtered_data = []
for elem in tweet_data_list: 
    filtered_tweet = clean_tweet(elem)
    filtered_data.append(filtered_tweet)
    
len(filtered_data)

9170

In [7]:
clean_tweets_json = []
for fd in filtered_data:
    #fd['created_at'] = datetime.strftime(fd['created_at'],"%Y-%m-%dT%H:%M:%S.%fZ")
    clean_tweets_json.append(fd)
with open("hmlscvd_clean.json", "w") as outfile:
    json.dump(clean_tweets_json, outfile, indent=4)

In [8]:
filtered_data = json.loads(open('/Users/planetyunnn/Desktop/HUSTLE/2022 Spring/COMM 557/Codes/Sentiment Analysis/hmlscvd_clean.json').read())

tweet_text = []
for tweet in filtered_data:
    if tweet["is_en"]:
        if tweet.get("is_rt"): 
            tweet_text.append(tweet["rt_text"].replace("\n", " "))
        else:
            tweet_text.append(tweet["text"].replace("\n", " "))
            
# There are fewer than 1,000 since filtered for english tweets
len(tweet_text)

9094

In [9]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ' ')    
    return text

def strip_mentions(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def strip_hashtags(text):
    entity_prefixes = ['#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)
        
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [10]:
stripped_tweet_text = []
for elem in tweet_text:
    elem = strip_links(elem)
    elem = strip_mentions(elem)
    elem = strip_hashtags(elem)
    elem = elem.replace('RT', '')
    elem = remove_special_characters(elem)
    stripped_tweet_text.append(elem)

In [11]:
for i,elem in enumerate(stripped_tweet_text):
    elem_textblob = TextBlob(elem)
    sent = elem_textblob.sentiment
    print (elem)
    print(sent)
    print ("----")

Heartless Tories laughing amp joking as they walk past the homeless amp Covid bereaved JohnsonOut70 JohnsonOut70 Vote Tactically to boot them out JohnsonOut70
Sentiment(polarity=-0.25, subjectivity=0.25)
----
This is why its confusing when school openers are called white supremacists For Black students the chronic absence rate is nearly 57 For Latinos it is 49 For homeless students it is 68  Nearly half of L A students chronically absent amid COVID
Sentiment(polarity=-0.09097222222222222, subjectivity=0.23333333333333336)
----
Heartless Tories laughing amp joking as they walk past the homeless amp Covid bereaved JohnsonOut70 JohnsonOut70 Vote Tactically to boot them out JohnsonOut70
Sentiment(polarity=-0.25, subjectivity=0.25)
----
Heartless Tories laughing amp joking as they walk past the homeless amp Covid bereaved JohnsonOut70 JohnsonOut70 Vote Tactically to boot them out JohnsonOut70
Sentiment(polarity=-0.25, subjectivity=0.25)
----
Heartless Tories laughing amp joking as they walk

Sentiment(polarity=-0.0619047619047619, subjectivity=0.44761904761904764)
----
Pastor Derek Remer who like Pastor Artur was ticketed and targeted throughout Covid 19 restrictions for feeding the homeless was just arrested Police selectively enforced the protest injuction against amplification to justify the arrest More soon at
Sentiment(polarity=0.225, subjectivity=0.25)
----
Pastor Derek Remer who like Pastor Artur was ticketed and targeted throughout Covid 19 restrictions for feeding the homeless was just arrested Police selectively enforced the protest injuction against amplification to justify the arrest More soon at
Sentiment(polarity=0.225, subjectivity=0.25)
----
Pastor Derek Remer who like Pastor Artur was ticketed and targeted throughout Covid 19 restrictions for feeding the homeless was just arrested Police selectively enforced the protest injuction against amplification to justify the arrest More soon at
Sentiment(polarity=0.225, subjectivity=0.25)
----
Pastor Derek Remer wh

Its hard to find a positive at the moment Homeless deaths 80 in2 years Huge Covid death rate avoidable Pre Lockdown Austerity deaths 140K BMJ 8 5K people died in 2021 through cold homes More will die and they dont give a fuck Evil ToryScum Greedy bastards 
Sentiment(polarity=-0.16634199134199132, subjectivity=0.7267316017316017)
----
when the government chose to end homelessness because of Covid they just did it This is deliberate premeditated murder of the most vulnerable people in society
Sentiment(polarity=0.0, subjectivity=0.5)
----
when the government chose to end homelessness because of Covid they just did it This is deliberate premeditated murder of the most vulnerable people in society
Sentiment(polarity=0.0, subjectivity=0.5)
----
Homeless deaths in the UK have increased by 80 since 2019 When covid hit we saw that with the political will we could end homelessness  yet we chose not to Snow on the streets in MCR and roughsleepers everywhere
Sentiment(polarity=0.0, subjectivity=0

Sentiment(polarity=0.4222222222222222, subjectivity=0.5166666666666667)
----
In Ireland there s an energy crisis housing crisis homeless crisis poverty crisis hospitals amp schools are full panic over rising covid cases and yet politicians are fine with housing over 200 000 mostly unvaxxed refugees of all nationalities at the taxpayers expense How
Sentiment(polarity=0.4222222222222222, subjectivity=0.5166666666666667)
----
In Ireland there s an energy crisis housing crisis homeless crisis poverty crisis hospitals amp schools are full panic over rising covid cases and yet politicians are fine with housing over 200 000 mostly unvaxxed refugees of all nationalities at the taxpayers expense How
Sentiment(polarity=0.4222222222222222, subjectivity=0.5166666666666667)
----
When organization try to let they voice be heard I just think its Media Documents to get funding We seen this Show at the Beginning of the Covid Pandemic Cause me and you know who was the first to hashtag Homeless can t sta

How has Covid 19 impacted the social welfare activities of Tokyo based Buddhist organisations Join us to hear baffelli talk about being with the volunteers and the experiences of precarity amp marginalisation in Tokyo homeless communities 
Sentiment(polarity=0.03333333333333333, subjectivity=0.06666666666666667)
----
Israel is still demolishing Palestinian homes and burning down olive trees in order to confiscate their land leaving them homeless and poor while they are supposed to stay home to protect themselves from Covid 19 PalestineLandDay
Sentiment(polarity=-0.2777777777777778, subjectivity=0.4444444444444444)
----
Stop getting The US involved with your corruption House the homeless vets on the streets in the US amp worry about America 1st You are a disgrace amp the worst President in History I want my vote back amp my life back End this Covid BS I can t wait until November 2022
Sentiment(polarity=-0.3333333333333333, subjectivity=0.3333333333333333)
----
Its lucky that America alr

...

waiting for liberals to explain Where the flu went in 2020 Why the global death count didn t change in 2020 Why Covid didn t wipe out the homeless population Why billions of healthy people were quarantined for the first time in history Why Covid avoided Africa Answers
Sentiment(polarity=0.25, subjectivity=0.27777777777777773)
----
Never heard so many lies in my life Like the non existent Covid that has ruined so many lives left people homeless but you Mr President who are part of the New World order just do as they instruct Take hold of this beautiful country the beautiful people and do different things
Sentiment(polarity=0.4051948051948052, subjectivity=0.5792207792207792)
----
I raise for the Leukemia amp Lymphoma Society every year Pre COVID I volunteered with a group that threw birthday parties for homeless kids I delivered candy to the kids in the hospital on holidays to name a few Your turn what are YOU doing to make the world a better place
Sentiment(polarity=0.15, subjectivity=

OMG Biden s inflation is tanking the world economy All because he kept poor Americans from becoming homeless panhandlers And foolishly tried to deny us the right to kill ourselves and our closest with COVID What a Loser
Sentiment(polarity=0.1119047619047619, subjectivity=0.6619047619047619)
----
OMG Biden s inflation is tanking the world economy All because he kept poor Americans from becoming homeless panhandlers And foolishly tried to deny us the right to kill ourselves and our closest with COVID What a Loser
Sentiment(polarity=0.1119047619047619, subjectivity=0.6619047619047619)
----
OMG Biden s inflation is tanking the world economy All because he kept poor Americans from becoming homeless panhandlers And foolishly tried to deny us the right to kill ourselves and our closest with COVID What a Loser
Sentiment(polarity=0.1119047619047619, subjectivity=0.6619047619047619)
----
OMG Biden s inflation is tanking the world economy All because he kept poor Americans from becoming homeless 

Nearly 10 of houses in the U S sit empty Thats 16 million empty homes compared to 550 000 homeless people But were letting our elders sleep and die on the streets
Sentiment(polarity=-0.03333333333333333, subjectivity=0.4666666666666666)
----
Nearly 10 of houses in the U S sit empty Thats 16 million empty homes compared to 550 000 homeless people But were letting our elders sleep and die on the streets
Sentiment(polarity=-0.03333333333333333, subjectivity=0.4666666666666666)
----
Nearly 10 of houses in the U S sit empty Thats 16 million empty homes compared to 550 000 homeless people But were letting our elders sleep and die on the streets
Sentiment(polarity=-0.03333333333333333, subjectivity=0.4666666666666666)
----
Nearly 10 of houses in the U S sit empty Thats 16 million empty homes compared to 550 000 homeless people But were letting our elders sleep and die on the streets
Sentiment(polarity=-0.03333333333333333, subjectivity=0.4666666666666666)
----
Nearly 10 of houses in the U S s

He called tip line to turn himself in amp your cops never showed up Blocks away they were assaulting the homeless Your 1000 cops on subway detail didn t stop him photograph him or halt train lines Some guy Zack got him You spread Covid at a film festival for the disabled
Sentiment(polarity=-0.2, subjectivity=0.3)
----
He called tip line to turn himself in amp your cops never showed up Blocks away they were assaulting the homeless Your 1000 cops on subway detail didn t stop him photograph him or halt train lines Some guy Zack got him You spread Covid at a film festival for the disabled
Sentiment(polarity=-0.2, subjectivity=0.3)
----
He called tip line to turn himself in amp your cops never showed up Blocks away they were assaulting the homeless Your 1000 cops on subway detail didn t stop him photograph him or halt train lines Some guy Zack got him You spread Covid at a film festival for the disabled
Sentiment(polarity=-0.2, subjectivity=0.3)
----
He called tip line to turn himself in am

11 11 If Covid is that dangerous why aren t millions of homeless dying on the streets
Sentiment(polarity=-0.6, subjectivity=0.9)
----
Did not make it to my doctors appointment for Monday Late on my insurance bill Low on insulin I currently have covid Im homeless No shelter I need help raising money for a vehicle aka shelter I am desperate for help CA teneshadoesart GFM
Sentiment(polarity=-0.22499999999999998, subjectivity=0.575)
----
11 11 If Covid is that dangerous why aren t millions of homeless dying on the streets
Sentiment(polarity=-0.6, subjectivity=0.9)
----
Meanwhile the Mayor of fun currently sidelined by COVID after popping bottles at the club whose great innovative public safety plan was to throw homeless ppls belongings in sanitation trucks now gets credited by pundits for being a tough on crime zealot Unbelievable
Sentiment(polarity=0.1373015873015873, subjectivity=0.6071428571428571)
----
Did not make it to my doctors appointment for Monday Late on my insurance bill Low o

In [12]:
text_sentiment = []

for i,elem in enumerate(stripped_tweet_text):
    elem_textblob = TextBlob(elem)
    sent = elem_textblob.sentiment
    text_sentiment.append(sent)

tblb_df = pd.DataFrame(text_sentiment)

#get the means
pol_var = tblb_df['polarity'].var()
sub_var = tblb_df['subjectivity'].var()
pol_avg = tblb_df['polarity'].mean()
sub_avg = tblb_df['subjectivity'].mean()

print([pol_var, sub_var])
print([pol_avg, sub_avg])

[0.06014406232251233, 0.0482004752106897]
[0.057604979190190225, 0.38119511866821937]


In [13]:
analyser = SentimentIntensityAnalyzer()

tweet_sentiment = []

for elem in stripped_tweet_text:
    #tweet_sentiment.append(elem)
    tweet_sentiment.append(analyser.polarity_scores(elem))

sentiment_df = pd.DataFrame(tweet_sentiment)
#print(sentiment_df)

#get the means
neg_avg = sentiment_df['neg'].mean()
neu_avg = sentiment_df['neu'].mean()
pos_avg = sentiment_df['pos'].mean()
com_avg = sentiment_df['compound'].mean()

neg_var = sentiment_df['neg'].var()
neu_var = sentiment_df['neu'].var()
pos_var = sentiment_df['pos'].var()
com_var = sentiment_df['compound'].var()

print([neg_avg, neu_avg, pos_avg, com_avg])
print([neg_var, neu_var, pos_var, com_var])

[0.1269694303936671, 0.8181775896195201, 0.05478524301737466, -0.2983202551132527]
[0.012418075167892626, 0.015207102627599806, 0.005037054171297854, 0.2735957267556283]


In [24]:
# Stemming / Lemming

### loading a spacy language model
# python -m spacy download en_core_web_sm
# https://spacy.io/models/en
nlp = spacy.load('en_core_web_sm') 

def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/planetyunnn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [26]:
words_corpus = []
for elem in stripped_tweet_text:
    # remove stop words
    elem = remove_stopwords(elem)
    # lemmatize text
    elem = lemmatize_text(elem)
    words_corpus.append(elem.lower().split())
print(len(words_corpus))

dictionary = corpora.Dictionary(words_corpus)
print(len(dictionary))

9094
7431


In [27]:
dictionary = corpora.Dictionary(words_corpus)
dictionary.filter_extremes(no_below=125, no_above=0.2, keep_n=10000)

corpus_bow = [dictionary.doc2bow(text) for text in words_corpus]

from gensim import corpora, models

tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

In [29]:
num_topics = 8
lda_model_tfidf = models.LdaMulticore(corpus_tfidf, num_topics=num_topics, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.056*"empty" + 0.030*"nearly" + 0.029*"u" + 0.029*"000" + 0.028*"house" + 0.028*"compare" + 0.028*"550" + 0.028*"16" + 0.028*"sit" + 0.028*"that"
Topic: 1 Word: 0.022*"2020" + 0.019*"go" + 0.018*"pastor" + 0.017*"19" + 0.017*"amp" + 0.015*"arrest" + 0.014*"like" + 0.014*"crime" + 0.014*"public" + 0.013*"time"
Topic: 2 Word: 0.042*"anymore" + 0.042*"convert" + 0.041*"hq" + 0.041*"musk" + 0.041*"elon" + 0.041*"corporate" + 0.041*"something" + 0.041*"big" + 0.041*"post" + 0.040*"city"
Topic: 3 Word: 0.029*"line" + 0.025*"cop" + 0.020*"community" + 0.020*"call" + 0.016*"2" + 0.016*"spread" + 0.015*"1000" + 0.015*"guy" + 0.015*"got" + 0.015*"turn"
Topic: 4 Word: 0.030*"need" + 0.023*"i" + 0.018*"hotel" + 0.017*"insulin" + 0.016*"two" + 0.016*"m" + 0.015*"ca" + 0.015*"raise" + 0.015*"week" + 0.014*"help"
Topic: 5 Word: 0.022*"50" + 0.020*"health" + 0.019*"find" + 0.018*"family" + 0.018*"russia" + 0.018*"crisis" + 0.018*"group" + 0.017*"loss" + 0.017*"permanent" + 0.017*"sudde